In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


/home/menion/anaconda3/envs/ai/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
element_size = 28
time_steps = 28
num_classes = 10
batch_size = 128
hidden_layer_size = 128

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
_inputs = tf.placeholder(tf.float32,shape=[None, time_steps,element_size],name='inputs')
y = tf.placeholder(tf.float32, shape=[None, num_classes],name='inputs')

# TensorFlow built-in functions
rnn_cell = tf.contrib.rnn.BasicRNNCell(hidden_layer_size)
outputs, _ = tf.nn.dynamic_rnn(rnn_cell, _inputs, dtype=tf.float32)

Wl = tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes],mean=0,stddev=.01))
bl = tf.Variable(tf.truncated_normal([num_classes],mean=0,stddev=.01))


In [4]:
def get_linear_layer(vector):
    return tf.matmul(vector, Wlget_linear_layer) + bl

In [5]:
# Build the output layer
last_rnn_output = outputs[:,-1,:]
final_output = get_linear_layer(last_rnn_output)
softmax = tf.nn.softmax_cross_entropy_with_logits(logits=final_output,labels=y)

# Build the function to minimize
cross_entropy = tf.reduce_mean(softmax)

# Build the optimizier
train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)

# Build the test computation
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(final_output,1))
accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32)))*100

# Use an interactive session and initialize the variables
sess=tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

test_data = mnist.test.images[:batch_size].reshape((-1,time_steps, element_size))
test_label = mnist.test.labels[:batch_size]


In [6]:
# Train for 3001 batches of size batch_size
for i in range(3001):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    batch_x = batch_x.reshape((batch_size, time_steps, element_size))
    sess.run(train_step,feed_dict={_inputs:batch_x, y:batch_y})
    # Print the accuracy every 1k batches
    if i % 1000 == 0:
        acc = sess.run(accuracy, feed_dict={_inputs: batch_x, y: batch_y})
        loss = sess.run(cross_entropy,feed_dict={_inputs:batch_x, y:batch_y})
        print ("Iter " + str(i) + ", Minibatch Loss= " + \
        "{:.6f}".format(loss) + ", Training Accuracy= " + \
        "{:.5f}".format(acc))
        
# Print the final accuracy
print ("Testing Accuracy:", sess.run(accuracy, feed_dict={_inputs: test_data, y: test_label}))

Iter 0, Minibatch Loss= 2.301724, Training Accuracy= 9.37500
Iter 1000, Minibatch Loss= 0.501239, Training Accuracy= 89.84375
Iter 2000, Minibatch Loss= 0.089079, Training Accuracy= 98.43750
Iter 3000, Minibatch Loss= 0.135005, Training Accuracy= 95.31250
Testing Accuracy: 98.4375
